<a href="https://colab.research.google.com/github/Spykabore15/students_RAG_project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive and Install Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q \
  langchain \
  langchain-community==0.2.* \
  faiss-cpu \
  sentence-transformers \
  transformers \
  accelerate \
  pypdf==4.* \
  streamlit \
  pyngrok


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 158.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 152.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency re

### Set up ngrok

In [ ]:
# Set up ngrok
from pyngrok import ngrok
from google.colab import userdata
import os

# Get ngrok token from secrets
ngrok_token = userdata.get('NGROK_AUTH_KEY')
ngrok.set_auth_token(ngrok_token)


### Streamit app configuration

In [ ]:
%%writefile /content/app.py
import streamlit as st
from pathlib import Path
import json
from typing import List
import torch

# Langchain imports
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline

# Page config
st.set_page_config(
    page_title="Student RAG Chatbot",
    page_icon="🎓",
    layout="wide"
)

# Constants
DATA_PATH = "/content/drive/MyDrive/TRAINING DATA"

@st.cache_resource
def load_core_pdfs(data_path: Path) -> List[Document]:
    """Load core PDF documents with caching"""
    core_pdfs = [
        data_path / "Projects.pdf",
        data_path / "criteria.pdf",
        data_path / "Mentors.pdf"
    ]

    docs = []
    for pdf in core_pdfs:
        if pdf.exists():
            pages = PyPDFLoader(str(pdf)).load()
            for p in pages:
                p.metadata.update({"source": pdf.name, "category": "core"})
            docs.extend(pages)
    return docs

@st.cache_resource
def load_student_dirs(data_path: Path) -> List[Document]:
    """Load student directories with caching"""
    docs = []
    students_dir = data_path / "students"

    if not students_dir.exists():
        return docs

    for student_dir in students_dir.iterdir():
        if not student_dir.is_dir():
            continue

        # Load student report
        report_pdf = student_dir / "report.pdf"
        if report_pdf.exists():
            pages = PyPDFLoader(str(report_pdf)).load()
            for p in pages:
                p.metadata.update({
                    "source": f"{student_dir.name}/report.pdf",
                    "student": student_dir.name,
                    "category": "student_report"
                })
            docs.extend(pages)

        # Load student summary
        summary_txt = student_dir / "summary.txt"
        if summary_txt.exists():
            tdocs = TextLoader(str(summary_txt), encoding="utf-8").load()
            for d in tdocs:
                d.metadata.update({
                    "source": f"{student_dir.name}/summary.txt",
                    "student": student_dir.name,
                    "category": "student_summary"
                })
            docs.extend(tdocs)

        # Load student metadata
        meta_json = student_dir / "metadata.json"
        if meta_json.exists():
            try:
                meta = json.loads(meta_json.read_text(encoding="utf-8"))
                meta_doc = Document(
                    page_content=json.dumps(meta, ensure_ascii=False, indent=2),
                    metadata={
                        "source": f"{student_dir.name}/metadata.json",
                        "student": student_dir.name,
                        "category": "student_metadata"
                    }
                )
                docs.append(meta_doc)
            except Exception as e:
                st.warning(f"Couldn't parse {meta_json}: {e}")

    return docs

@st.cache_resource
def initialize_rag_chain():
    """Initialize the complete RAG chain with caching"""
    import torch

    # Check GPU availability
    device = "cuda" if torch.cuda.is_available() else "cpu"
    st.info(f"Using device: {device}")

    if device == "cpu":
        st.warning("⚠️ GPU not detected! This will be very slow. Make sure GPU is enabled in Colab.")

    with st.spinner("Loading documents..."):
        # Load documents
        data_path = Path(DATA_PATH)
        raw_docs = load_core_pdfs(data_path) + load_student_dirs(data_path)
        st.success(f"✓ Loaded {len(raw_docs)} documents")

    with st.spinner("Splitting documents into chunks..."):
        # Split documents
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks = splitter.split_documents(raw_docs)
        st.success(f"✓ Created {len(chunks)} chunks")

    with st.spinner("Loading embedding model..."):
        # Initialize embeddings with GPU
        embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            model_kwargs={'device': device}
        )
        st.success("✓ Embeddings model loaded")

    with st.spinner("Building vector store..."):
        # Create vector store
        vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=embeddings
        )
        st.success("✓ Vector store created")

    with st.spinner("Loading language model (this may take a few minutes)..."):
        # Initialize LLM pipeline with GPU and optimizations

        pipe = pipeline(
            "text-generation",
            model="mistralai/Mistral-7B-Instruct-v0.2",
            device=0 if device == "cuda" else -1,  # 0 = GPU, -1 = CPU
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            temperature=0.001,
            max_new_tokens=200,
            return_full_text=False
        )
        llm = HuggingFacePipeline(pipeline=pipe)
        st.success(f"✓ Language model loaded on {device}")

    # Create retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}
    )

    # Define prompt template
    template = """
You are a helpful assistant that answers using only the provided context.
If the answer is not contained in the context, say "I do not know".

Context: {context}

Question: {question}

Other requirement: Structure the answer in a way that is easy to read and understand.
"""

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    # Create chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    return chain, retriever

# Main app
def main():
    st.title("🎓 Student Project RAG Chatbot")
    st.markdown("Ask questions about student projects, mentors, and evaluation criteria.")

    # Initialize chain (cached)
    try:
        chain, retriever = initialize_rag_chain()
    except Exception as e:
        st.error(f"Error initializing RAG system: {e}")
        st.stop()

    # Sidebar
    with st.sidebar:
        st.header("Settings")
        show_sources = st.checkbox("Show source documents", value=True)
        st.markdown("---")
        st.markdown("### Example Questions")
        st.markdown("""
        - Who are the mentors for robotics projects?
        - Which students worked on AI healthcare projects?
        - What are the evaluation criteria?
        - Provide names of students for a Robot and Air Quality project
        """)

    # Chat interface
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if "sources" in message and show_sources:
                with st.expander("📚 View Sources"):
                    st.markdown(message["sources"])

    # Chat input
    if question := st.chat_input("Ask a question about student projects..."):
        # Add user message
        st.session_state.messages.append({"role": "user", "content": question})
        with st.chat_message("user"):
            st.markdown(question)

        # Get response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                try:
                    result = chain(question)
                    answer = result["result"]

                    # Display answer
                    st.markdown(answer)

                    # Format sources
                    sources_text = ""
                    if show_sources and "source_documents" in result:
                        sources_text = "\n\n**Retrieved Sources:**\n\n"
                        for i, doc in enumerate(result["source_documents"], 1):
                            source = doc.metadata.get('source', 'Unknown')
                            category = doc.metadata.get('category', 'N/A')
                            sources_text += f"{i}. **{source}** ({category})\n"
                            sources_text += f"   {doc.page_content[:200]}...\n\n"

                        with st.expander("📚 View Sources"):
                            st.markdown(sources_text)

                    # Add to chat history
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": answer,
                        "sources": sources_text
                    })

                except Exception as e:
                    error_msg = f"Error generating response: {e}"
                    st.error(error_msg)
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": error_msg
                    })

    # Clear chat button
    if st.sidebar.button("🗑️ Clear Chat History"):
        st.session_state.messages = []
        st.rerun()

if __name__ == "__main__":
    main()

Writing /content/app.py


### Run the app with ngrok

In [ ]:

import subprocess
import time

# Kill any existing streamlit processes
!pkill -f streamlit

# Start Streamlit in background
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "/content/app.py", "--server.port=8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to start
time.sleep(10)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print("=" * 50)
print(f"Streamlit app is running!")
print(f"Public URL: {public_url}")
print("=" * 50)
print("\nClick the URL above to access your app")
print("Keep this cell running to maintain the connection")

Streamlit app is running!
Public URL: NgrokTunnel: "https://unmustered-lacresha-tetratomic.ngrok-free.dev" -> "http://localhost:8501"

Click the URL above to access your app
Keep this cell running to maintain the connection
